# This notebook demonstrates the application of our estimator to the fruit fly genetics data from Hao et. al. (2008)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import csv
import scipy.stats
from utils import get_counts, construct_A_gaussian, construct_A_gaussian_pdf
from otherEstimators import mle
from FWER_utils import estimateZeta_FWER_manyThresholds
import scipy
import pickle
from multiprocessing import Pool

In [ ]:
# load raw data
X = []
k=0
l=0
# see README.txt for explanation of data. Here we extract two Z-scores per gene (there are 13071 genes)
with open('flyData.txt', newline='') as csvfile:
    spamreader = csv.DictReader(csvfile,delimiter='\t')
    print(spamreader.fieldnames)
    for row in spamreader:
        if row['target_name']=='empty':
            k +=1
        else:
            l += 1
            first_Z_score = float(row['1 z w/o controls'])
            second_Z_score = float(row['2 z w/o controls'])
            X.append([first_Z_score, second_Z_score ])
print(k,l)
X = -np.array(X) # flip sign so large observations are discoveries
print(X.shape)

mu_hat = np.mean(X, axis=1) # average the 2 Z-scores together
sigmaSquared = 0.25  # Our fitted variance; note that the observed distribution is a poor match for the theoretical N(0, 1/2)

mu_hat_tStats = mu_hat/np.sqrt(2)  # To generated a t-statistic, divide by the square root of the number of replicates

In [ ]:
#fig, ax = plt.subplots(figsize=(5,4))  # Size for the paper
fig, ax = plt.subplots(figsize=(7,4))
plt.rcParams["font.size"] = 16
ax = plt.subplot(111)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['left'].set_visible(False)
frame1 = plt.gca()
frame1.axes.get_yaxis().set_visible(False)

grid = np.linspace(min(mu_hat), max(mu_hat), 1000)
plt.plot(grid, get_counts(mu_hat, grid))
#plt.title("Distribution of Z-scores for the fly data")
plt.xlabel("Z-score")
fwer_cutoff = scipy.stats.norm.ppf(1-0.05/len(mu_hat), scale=np.sqrt(sigmaSquared))
plt.vlines(fwer_cutoff, ymin=0, ymax=120, linestyle='--', label="Bonferroni\ncorrected\ncritical value")
plt.legend()
plt.tight_layout(pad=2)
plt.savefig("zscoresFly-withBF.png")
plt.savefig("zscoresFly-withBF.eps")
plt.savefig("zscoresFly-withBF.pdf")

In [ ]:
# Fit the MLE with a less fine grid, to save computation
grid_mle = np.linspace(min(mu_hat), max(mu_hat), 1000)
b = get_counts(mu_hat, grid_mle)
b = b/sum(b)

A = construct_A_gaussian_pdf(grid_mle, sigmaSquared=sigmaSquared)

In [ ]:
w_mle, _ = mle(A, b, max_iters=10000, verbose=True)

In [ ]:
plt.plot(grid_mle, 85*get_counts(mu_hat, grid_mle)/np.sum(get_counts(mu_hat, grid_mle)))
plt.plot(grid_mle, A@w_mle, linewidth=4)
plt.show()

plt.plot(grid_mle, w_mle)

In [ ]:
grid = np.linspace(min(mu_hat), max(mu_hat), 1000)

zetaHats = estimateEntireLine(obs=mu_hat, 
                              tolerance=1/len(mu_hat), 
                              alpha=0.05,
                              hypTest=KS_test,
                              grid=grid,
                              gammas=np.linspace(0, 1, 20),
                              sigma=np.sqrt(sigmaSquared),
                              verbose=False)

In [ ]:
NUM_CORES = 10
zetaHats_FWER = estimateZeta_FWER_manyThresholds_parallel(obs=mu_hat, gammas=np.linspace(0, 1, 20),
                                                          alpha=0.05, sigma=np.sqrt(sigmaSquared),
                                                          distribution="normal", numCores=NUM_CORES)

In [ ]:
fig, ax = plt.subplots(figsize=(7,4))
ax = plt.subplot(111)
plt.gcf().subplots_adjust(bottom=0.15, left=0.25)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

modGrid = list(np.linspace(0, 1, 20))+[3]
zetaHatsMod = list(zetaHats)+[0]
print(modGrid)
c=380
cMax = 530
plt.plot(grid_mle[c:cMax], 1-np.cumsum(w_mle/np.sum(w_mle))[c:cMax], label=r'$\widehat{\zeta}_{MLE}$', 
         linestyle="-", linewidth=2, color="darkgreen")
plt.plot(modGrid, zetaHatsMod, label=r'$\widehat{\zeta}_{KS}$ (ours)', linestyle="--", linewidth=2, color="purple" )
#plt.title("Our estimator and the MLE plug-in estimate")
plt.plot(modGrid, list(zetaHats_FWER)+[0], label=r'$\widehat{\zeta}_{FWER}$ (identification)', color="red", linestyle=":", linewidth=2)
plt.plot()
plt.ylabel("Fraction of mass\nabove threshold")
plt.xlabel("Effect Size")
plt.legend()
plt.tight_layout(pad=2)
plt.savefig("MLE-fly.png")
plt.savefig("MLE-fly.eps")
plt.savefig("MLE-fly.pdf")
plt.show()